## Tăng cường dữ liệu

In [3]:
import cv2
import albumentations as A
import os
from pathlib import Path

# Thư mục chứa 5 ảnh gốc và thư mục lưu ảnh tăng cường
input_dir = "input_images"  # 5 ảnh gốc: person1.jpg, person2.jpg, ..., person5.jpg
output_dir = "augmented_images"
os.makedirs(output_dir, exist_ok=True)

# Định nghĩa pipeline tăng cường dữ liệu
transform = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Xoay ngẫu nhiên ±30 độ
    A.HorizontalFlip(p=0.5),  # Lật ngang
    A.RandomBrightnessContrast(p=0.5),  # Thay đổi độ sáng/tương phản
    A.GaussNoise(p=0.3),  # Thêm nhiễu
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Resize(height=224, width=224),  # Resize về kích thước thống nhất
])

# Danh sách người và nhãn
people = ["QuanPhan", "DangDuong", "ManhTuong", "HuuTho"]

# Tăng cường dữ liệu
num_augmented_per_image = 200  # Mỗi ảnh gốc tạo 200 ảnh mới
for person in people:
    img_path = os.path.join(input_dir, f"{person}.jpg")
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển sang RGB
    
    # Tạo thư mục cho mỗi người
    person_dir = os.path.join(output_dir, person)
    os.makedirs(person_dir, exist_ok=True)
    
    # Tạo ảnh tăng cường
    for i in range(num_augmented_per_image):
        augmented = transform(image=img)["image"]
        save_path = os.path.join(person_dir, f"{person}_aug_{i}.jpg")
        cv2.imwrite(save_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

print(f"Đã tạo {num_augmented_per_image * len(people)} ảnh tăng cường.")

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\core\validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Đã tạo 800 ảnh tăng cường.


In [5]:
import os
import shutil
from pathlib import Path
import random

# Thư mục chứa ảnh tăng cường và thư mục đầu ra
augmented_dir = "augmented_images"
output_base_dir = "dataset"
train_dir = os.path.join(output_base_dir, "train")
val_dir = os.path.join(output_base_dir, "val")
test_dir = os.path.join(output_base_dir, "test")

# Tỷ lệ chia tập
train_ratio = 0.7  # 70% cho train
val_ratio = 0.15   # 15% cho validation
test_ratio = 0.15  # 15% cho test

# Danh sách người
people = ["QuanPhan", "DangDuong", "ManhTuong", "HuuTho"]

# Tạo thư mục train, val, test
for split_dir in [train_dir, val_dir, test_dir]:
    for person in people:
        os.makedirs(os.path.join(split_dir, person), exist_ok=True)

# Hàm chia dữ liệu
for person in people:
    # Lấy danh sách tất cả ảnh của person
    person_dir = os.path.join(augmented_dir, person)
    images = [f for f in os.listdir(person_dir) if f.endswith(".jpg")]
    random.shuffle(images)  # Xáo trộn ngẫu nhiên

    # Tính số lượng ảnh cho từng tập
    total_images = len(images)
    train_count = int(total_images * train_ratio)
    val_count = int(total_images * val_ratio)
    test_count = total_images - train_count - val_count  # Đảm bảo tổng = 100%

    # Chia danh sách ảnh
    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Sao chép ảnh vào các thư mục tương ứng
    for img in train_images:
        shutil.copy(os.path.join(person_dir, img), os.path.join(train_dir, person, img))
    for img in val_images:
        shutil.copy(os.path.join(person_dir, img), os.path.join(val_dir, person, img))
    for img in test_images:
        shutil.copy(os.path.join(person_dir, img), os.path.join(test_dir, person, img))

    print(f"Person {person}:")
    print(f"  Train: {len(train_images)} images")
    print(f"  Val: {len(val_images)} images")
    print(f"  Test: {len(test_images)} images")

Person QuanPhan:
  Train: 140 images
  Val: 30 images
  Test: 30 images
Person DangDuong:
  Train: 140 images
  Val: 30 images
  Test: 30 images
Person ManhTuong:
  Train: 140 images
  Val: 30 images
  Test: 30 images
Person HuuTho:
  Train: 140 images
  Val: 30 images
  Test: 30 images


# Huấn luyện

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import matplotlib.pyplot as plt
import onnxruntime as ort
import os
from datetime import datetime
import numpy as np
import tf2onnx
import onnx


In [8]:
# Cấu hình siêu tham số
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
FINE_TUNE_LAYERS = 4  # Số tầng cuối của VGG16 để tinh chỉnh
DATA_DIR = "dataset"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")


# Kiểm tra thư mục dữ liệu
for directory in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Thư mục không tồn tại: {directory}")
    
# Tạo mô hình VGG16
def build_model(num_classes=4):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Đóng băng tất cả tầng của VGG16 ban đầu
    for layer in base_model.layers:
        layer.trainable = False
    
    return model

# Chuẩn bị dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=20.0  # Thêm thay đổi màu sắc ngẫu nhiên
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


# Tạo và biên dịch mô hình
model = build_model(num_classes=4)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Callbacks
log_dir = f"logs/fit/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_vgg_face.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1),
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

# Huấn luyện giai đoạn 1: Chỉ huấn luyện các tầng đầu
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

# Giai đoạn 2: Tinh chỉnh các tầng cuối của VGG16
for layer in model.layers[-FINE_TUNE_LAYERS:]:
    layer.trainable = True

# Biên dịch lại với tốc độ học nhỏ hơn
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE / 10),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Tiếp tục huấn luyện
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)


# Save final model (sau khi kết thúc training)
model.save("vgg_face_final.h5")  # định dạng thư mục

# Chuyển đổi và lưu mô hình sang ONNX
model_proto, _ = tf2onnx.convert.from_keras(model, output_path="vgg_face_final.onnx")
onnx.save(model_proto, "vgg_face_final.onnx")

# Đánh giá trên tập kiểm tra
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_generator)
print(f"\nKết quả trên tập kiểm tra:")
print(f"Độ mất mát: {test_loss:.4f}")
print(f"Độ chính xác: {test_acc:.4f}")
print(f"Độ chính xác từng lớp: {test_precision:.4f}")
print(f"Độ phủ: {test_recall:.4f}")

# Vẽ và lưu biểu đồ
# Kết hợp lịch sử huấn luyện từ cả hai giai đoạn
metrics = ['accuracy', 'loss', 'precision', 'recall']
full_history = {}
for metric in metrics:
    full_history[metric] = history.history[metric] + history_fine.history[metric]
    full_history[f'val_{metric}'] = history.history[f'val_{metric}'] + history_fine.history[f'val_{metric}']

# Vẽ biểu đồ độ chính xác
plt.figure(figsize=(10, 5))
plt.plot(full_history['accuracy'], label='Độ chính xác huấn luyện')
plt.plot(full_history['val_accuracy'], label='Độ chính xác xác thực')
plt.title('Learning Curve - Độ chính xác')
plt.xlabel('Epoch')
plt.ylabel('Độ chính xác')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'accuracy_curve.png'))
plt.close()

# Vẽ biểu đồ mất mát
plt.figure(figsize=(10, 5))
plt.plot(full_history['loss'], label='Mất mát huấn luyện')
plt.plot(full_history['val_loss'], label='Mất mát xác thực')
plt.title('Learning Curve - Mất mát')
plt.xlabel('Epoch')
plt.ylabel('Mất mát')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'loss_curve.png'))
plt.close()

# Vẽ biểu đồ precision
plt.figure(figsize=(10, 5))
plt.plot(full_history['precision'], label='Precision huấn luyện')
plt.plot(full_history['val_precision'], label='Precision xác thực')
plt.title('Precision Curve')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'precision_curve.png'))
plt.close()

# Vẽ biểu đồ recall
plt.figure(figsize=(10, 5))
plt.plot(full_history['recall'], label='Recall huấn luyện')
plt.plot(full_history['val_recall'], label='Recall xác thực')
plt.title('Recall Curve')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'recall_curve.png'))
plt.close()




Found 560 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 5 classes.


f:\Study\Projects\HK6\XLA\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3792 - loss: 1.3928 - precision_2: 0.3977 - recall_2: 0.0917
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to best_vgg_face.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 134s 7s/step - accuracy: 0.3833 - loss: 1.3863 - precision_2: 0.4049 - recall_2: 0.0927 - val_accuracy: 0.6000 - val_loss: 0.9722 - val_precision_2: 1.0000 - val_recall_2: 0.2000 - learning_rate: 0.0010
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6597 - loss: 0.9588 - precision_2: 0.7886 - recall_2: 0.2861
Epoch 2: val_accuracy improved from 0.60000 to 0.93333, saving model to best_vgg_face.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 128s 7s/step - accuracy: 0.6640 - loss: 0.9534 - precision_2: 0.7938 - recall_2: 0.2912 - val_accuracy: 0.9333 - val_loss: 0.6393 - val_precision_2: 1.0000 - val_recall_2: 0.7000 - learning_rate: 0.0010
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8286 - loss: 0.6347 - precision_2: 0.9373 - recall_2: 0.6620
Epoch 3: val_accuracy improved from 0.93333 to 0.95000, saving model to best_vgg_face.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 129s 7s/step - accuracy: 0.8307 - loss: 0.6319 - precision_2: 0.9384 - recall_2: 0.6641 - val_accuracy: 0.9500 - val_loss: 0.4221 - val_precision_2: 1.0000 - val_recall_2: 0.8333 - learning_rate: 0.0010
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8940 - loss: 0.4568 - precision_2: 0.9645 - recall_2: 0.8123
Epoch 4: val_accuracy did not improve from 0.95000
18/18 ━━━━━━━━━━━━━━━━━━━━ 131s 7s/step - accuracy: 0.8944 - loss: 0.4558 - precision_2: 0.9650 - recall_2: 0.8129 - val_accuracy: 0.9333 - val_loss: 0.3304 - val_precision_2: 0.9815 - val_recall_2: 0.8833 - learning_rate: 0.0010
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9515 - loss: 0.3191 - precision_2: 0.9805 - recall_2: 0.8817
Epoch 5: val_accuracy improved from 0.95000 to 0.95833, saving model to best_vgg_face.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 109s 6s/step - accuracy: 0.9521 - loss: 0.3175 - precision_2: 0.9809 - recall_2: 0.8829 - val_accuracy: 0.9583 - val_loss: 0.2312 - val_precision_2: 1.0000 - val_recall_2: 0.9250 - learning_rate: 0.0010
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9549 - loss: 0.2532 - precision_2: 0.9926 - recall_2: 0.8942
Epoch 6: val_accuracy improved from 0.95833 to 0.96667, saving model to best_vgg_face.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 113s 6s/step - accuracy: 0.9546 - loss: 0.2536 - precision_2: 0.9920 - recall_2: 0.8942 - val_accuracy: 0.9667 - val_loss: 0.1908 - val_precision_2: 1.0000 - val_recall_2: 0.9333 - learning_rate: 0.0010
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9755 - loss: 0.2023 - precision_2: 0.9886 - recall_2: 0.9359
Epoch 7: val_accuracy improved from 0.96667 to 0.98333, saving model to best_vgg_face.h5


18/18 ━━━━━━━━━━━━━━━━━━━━ 117s 7s/step - accuracy: 0.9750 - loss: 0.2026 - precision_2: 0.9886 - recall_2: 0.9358 - val_accuracy: 0.9833 - val_loss: 0.1585 - val_precision_2: 0.9912 - val_recall_2: 0.9333 - learning_rate: 0.0010
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9777 - loss: 0.1721 - precision_2: 0.9871 - recall_2: 0.9590
Epoch 8: val_accuracy did not improve from 0.98333
18/18 ━━━━━━━━━━━━━━━━━━━━ 135s 8s/step - accuracy: 0.9774 - loss: 0.1716 - precision_2: 0.9871 - recall_2: 0.9592 - val_accuracy: 0.9833 - val_loss: 0.1271 - val_precision_2: 0.9915 - val_recall_2: 0.9667 - learning_rate: 0.0010
Epoch 9/50
11/18 ━━━━━━━━━━━━━━━━━━━━ 43s 6s/step - accuracy: 0.9795 - loss: 0.1356 - precision_2: 0.9935 - recall_2: 0.9656

KeyboardInterrupt: 

## Chuyển model .h5 sang onnx

In [6]:
import tensorflow as tf
import tf2onnx

# Tải mô hình
model = tf.keras.models.load_model("vgg_face_final")

# Lưu lại dưới dạng SavedModel
model.save("vgg_face_saved_model", save_format="tf")

# Chuyển đổi sang ONNX
onnx_model_path = "vgg_face_final.onnx"
!python -m tf2onnx.convert \
    --saved-model vgg_face_saved_model \
    --output {onnx_model_path} \
    --opset 13

INFO:tensorflow:Assets written to: vgg_face_saved_model\assets


INFO:tensorflow:Assets written to: vgg_face_saved_model\assets
<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2025-04-30 17:02:06,020 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2025-04-30 17:02:06,020 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2025-04-30 17:02:07,565 - INFO - Signatures found in model: [serving_default].
2025-04-30 17:02:07,565 - WARNING - '--signature_def' not specified, using first signature: serving_default
2025-04-30 17:02:07,565 - INFO - Output names: ['dense_1']
2025-04-30 17:02:10,864 - INFO - Using tensorflow=2.12.0, onnx=1.16.1, tf2onnx=1.16.1/15c810
2025-04-30 17:02:10,864 - INFO - Using opset <onnx, 13>
2025-04-30 17:02:11,306 - INFO - Computed 0 values for constant f

## Tải face_detector từ DNN-based Face Detector

In [1]:
import cv2
import onnx
import numpy as np

# Đường dẫn đến tệp Caffe
proto_path = r"F:\Study\Projects\HK6\XLA\Project\FaceDetection\deploy.prototxt"
model_path = r"F:\Study\Projects\HK6\XLA\Project\FaceDetection\res10_300x300_ssd_iter_140000_fp16.caffemodel"
output_onnx_path = r"F:\Study\Projects\HK6\XLA\Project\FaceDetection\face_detector.onnx"

# Tải mô hình Caffe
net = cv2.dnn.readNetFromCaffe(proto_path, model_path)

# Chuyển đổi sang ONNX
dummy_input = cv2.dnn.blobFromImage(np.zeros((300, 300, 3)), 1.0, (300, 300), (104.0, 177.0, 123.0))
net.setInput(dummy_input)
output = net.forward()
onnx_model = cv2.dnn.writeToONNX(net, dummy_input.shape)


# Lưu mô hình ONNX
with open(output_onnx_path, 'wb') as f:
    f.write(onnx_model.SerializeToString())

print(f"Model saved to {output_onnx_path}")

AttributeError: module 'cv2.dnn' has no attribute 'writeToONNX'

In [4]:
import numpy as np
import cv2
import onnxruntime as ort
import os

# Hàm phát hiện khuôn mặt sử dụng Haar Cascade
def detect_faces_haar(image, cascade_classifier, scale_factor=1.1, min_neighbors=5):
    # Chuyển ảnh sang grayscale (yêu cầu của Haar Cascade)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Phát hiện khuôn mặt
    faces = cascade_classifier.detectMultiScale(
        gray,
        scaleFactor=scale_factor,  # Tỷ lệ thu nhỏ mỗi lần quét
        minNeighbors=min_neighbors,  # Số lượng hàng xóm tối thiểu
        minSize=(30, 30)  # Kích thước tối thiểu của khuôn mặt
    )
    
    boxes = []
    for (x, y, w, h) in faces:
        x1, y1 = x, y
        x2, y2 = x + w, y + h
        boxes.append((x1, y1, x2, y2))
        print(f"Detected face: ({x1}, {y1}, {x2}, {y2})")
    
    return boxes

# Danh sách nhãn
class_names = ['Đẳng Cửu Dương', 'Hoàng Manh Tường', 'Quân Phan', 'Trịnh Hửu Thọ']

# 1. Load Haar Cascade Classifier
cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
if not os.path.exists(cascade_path):
    raise FileNotFoundError(f"Haar Cascade file not found: {cascade_path}")
face_cascade = cv2.CascadeClassifier(cascade_path)
if face_cascade.empty():
    raise ValueError("Failed to load Haar Cascade classifier")

# 2. Load VGG16 classifier
classifier_path = r"F:\Study\Projects\HK6\XLA\Project\FaceDetection\vgg_face_final.onnx"
if not os.path.exists(classifier_path):
    raise FileNotFoundError(f"ONNX file not found: {classifier_path}")
face_classifier = ort.InferenceSession(classifier_path)

# 3. Khởi tạo camera
cap = cv2.VideoCapture(0)  # 0 là camera mặc định
if not cap.isOpened():
    raise ValueError("Cannot open camera")

try:
    # 4. Vòng lặp xử lý khung hình từ camera
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Resize khung hình nếu quá lớn
        max_size = 1280
        if max(frame.shape[:2]) > max_size:
            scale = max_size / max(frame.shape[:2])
            frame = cv2.resize(frame, (int(frame.shape[1] * scale), int(frame.shape[0] * scale)))

        # Phát hiện khuôn mặt bằng Haar Cascade
        boxes = detect_faces_haar(frame, face_cascade, scale_factor=1.1, min_neighbors=5)

        # Lặp qua các bounding box và nhận diện
        for box in boxes:
            x1, y1, x2, y2 = box
            face_crop = frame[y1:y2, x1:x2]
            if face_crop.size == 0 or face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
                print("Skipping small face region")
                continue  # Bỏ qua vùng mặt quá nhỏ

            # Chuẩn bị ảnh cho phân loại
            face_crop = cv2.resize(face_crop, (224, 224))
            face_crop = face_crop[:, :, ::-1]  # BGR -> RGB
            face_crop = face_crop.astype(np.float32) / 255.0
            face_crop = np.expand_dims(face_crop, axis=0)

            # Phân loại khuôn mặt
            inputs = {face_classifier.get_inputs()[0].name: face_crop}
            preds = face_classifier.run(None, inputs)[0]
            label_id = np.argmax(preds)
            label_name = class_names[label_id]
            print(f"Detected: {label_name}, Box: ({x1}, {y1}, {x2}, {y2})")

            # Vẽ bounding box và nhãn
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Hiển thị khung hình
        cv2.imshow("Face Recognition", frame)

        # Thoát khi nhấn phím 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Giải phóng tài nguyên
    cap.release()
    cv2.destroyAllWindows()

Detected face: (240, 260, 429, 449)
Detected: Quân Phan, Box: (240, 260, 429, 449)
Detected face: (242, 258, 430, 446)
Detected: Quân Phan, Box: (242, 258, 430, 446)
Detected face: (241, 258, 432, 449)
Detected: Quân Phan, Box: (241, 258, 432, 449)
Detected face: (242, 258, 430, 446)
Detected: Quân Phan, Box: (242, 258, 430, 446)
Detected face: (240, 259, 427, 446)
Detected: Quân Phan, Box: (240, 259, 427, 446)
Detected face: (314, 255, 515, 456)
Detected: Trịnh Hửu Thọ, Box: (314, 255, 515, 456)
Detected face: (84, 128, 223, 267)
Detected face: (388, 247, 595, 454)
Detected: Trịnh Hửu Thọ, Box: (84, 128, 223, 267)
Detected: Hoàng Manh Tường, Box: (388, 247, 595, 454)
Detected face: (398, 246, 605, 453)
Detected face: (104, 131, 246, 273)
Detected: Quân Phan, Box: (398, 246, 605, 453)
Detected: Hoàng Manh Tường, Box: (104, 131, 246, 273)
Detected face: (110, 134, 259, 283)
Detected face: (399, 246, 605, 452)
Detected: Hoàng Manh Tường, Box: (110, 134, 259, 283)
Detected: Quân Phan, Box